<a href="https://colab.research.google.com/github/Abdillah021/Portofolio-data-analyst/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import string
import random

import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt_tab")
nltk.download("wordnet")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
data_root = '/content/drive/MyDrive/Data set/Time series/intents_medis.json'
data_file = open(data_root).read()
data = json.loads(data_file)
data

{'intents': [{'tag': 'salam_pagi',
   'patterns': ['pagi', 'halo', 'selamat pagi', 'good morning'],
   'responses': ['Selamat pagi! Apa keluhan yang sedang kamu alami?',
    'Pagi juga! Saya siap membantu keluhan kesehatanmu.'],
   'context': ''},
  {'tag': 'keluhan_pusing',
   'patterns': ['saya pusing',
    'kepala saya sakit',
    'pusing banget',
    'sakit kepala'],
   'responses': ['Coba istirahat sejenak dan pastikan kamu cukup minum air putih. Bila pusing berlanjut, sebaiknya konsultasi ke dokter.',
    'Pusing bisa disebabkan oleh kelelahan atau dehidrasi. Minumlah air dan hindari layar gadget sebentar.'],
   'context': ''},
  {'tag': 'keluhan_batuk',
   'patterns': ['saya batuk',
    'batuk terus',
    'tenggorokan gatal',
    'batuk gak berhenti'],
   'responses': ['Batuk bisa karena infeksi atau alergi. Coba minum air hangat dan hindari makanan berminyak.',
    'Jika batuk berlangsung lebih dari 3 hari, sebaiknya periksakan ke fasilitas kesehatan.'],
   'context': ''},
  {'

In [ ]:
#eksraksi data x dan y


words = [] # BOW model for pattern
classes = [] # bow for tag
data_x = [] # storing pattern
data_y = [] # storing tag

for intent in data["intents"]:
  for pattern in intent["patterns"]:
    tokens = nltk.word_tokenize(pattern)
    words.extend(tokens)
    data_x.append(pattern)
    data_y.append(intent["tag"])

    if intent["tag"] not in classes:
      classes.append(intent["tag"])

#lemmetizer
lemmatizer = WordNetLemmatizer()
#lowercase
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
#sorting
words = sorted(set(words))
classes = sorted(set(classes))
print(classes)
print(words)

['keluhan_batuk', 'keluhan_demam', 'keluhan_mual', 'keluhan_pusing', 'keluhan_sakit_perut', 'penutup', 'salam_pagi', 'ucapan_terima_kasih']
['badan', 'banget', 'batuk', 'berhenti', 'bye', 'dadah', 'demam', 'enak', 'gak', 'gatal', 'good', 'halo', 'jumpa', 'kasih', 'kepala', 'makasih', 'melilit', 'merasa', 'meriang', 'morning', 'mual', 'mule', 'muntah', 'naik', 'pagi', 'panas', 'pengen', 'perut', 'pusing', 'sakit', 'sampai', 'saya', 'sejak', 'selamat', 'suhu', 'tadi', 'tenggorokan', 'terima', 'terus', 'thank', 'thanks', 'tinggal', 'tubuh', 'ya', 'you']


In [ ]:
#text to nuumbers
training = []
out_empty = [0] * len(classes)
#model BOW
for idx, doc in enumerate(data_x):
  bow = []
  text = lemmatizer.lemmatize(doc.lower())
  for word in words:
    bow.append(1) if word in text else bow.append(0)
    #mark index ke current class pattern
    output_row = list(out_empty)
    output_row[classes.index(data_y[idx])] = 1
    #input ke training
    training.append([bow, output_row])
# comnvert ke array
random.shuffle(training)
training = np.array(training, dtype=object)
#split x dan y
train_x = np.array(list(training[:, 0]))
train_x
train_y = np.array(list(training[:, 1]))
train_y

array([[0, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
#model training Neural Network
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu"))
model.add(Dropout(0.5))
#dense itu buat menghububgkan tiap neuron dan 64 itu sel
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
# buat output neuron jadi probabiltas
# softmax untuk mebuat probabilistas 0-100%
model.add(Dense(len(train_y[0]), activation="softmax"))
# pelatihan model neural network dalam object optimization
# learning rate bisa susuaikan 0.01 sudah tinggi , decay 1e-6 '-6' untuk waktu lerning adam jangan terlalu lama
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
#menyusun MODEL NN sebelum di latih
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
# Model akan DI Training
# Fix: Use train_x and train_y instead of train_X and train_Y
model.fit(x=train_x, y=train_y, epochs=150, verbose=1)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         5,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8)              │           520 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,664 (57.28 KB)

 Trainable params: 14,664 (57.28 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/150
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5429 - loss: 1.3830
Epoch 2/150
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9890 - loss: 0.0504
Epoch 3/150
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9979 - loss: 0.0165
Epoch 4/150
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9996 - loss: 0.0060
Epoch 5/150
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9986 - loss: 0.0105
Epoch 6/150
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9975 - loss: 0.0100
Epoch 7/150
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9975 - loss: 0.0085
Epoch 8/150
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9986 - loss: 0.0099
Epoch 9/150
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9970 - loss: 0.0044
Epoch 10/150
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9996 - loss: 0.0025
Epoch 11/150
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9999 - loss: 0.0044
Epoch 12/150
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accu

In [ ]:
# PREPROCCESING INPUT
def clean_text(text):
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bag_of_words(text, vocab):
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens:
    for idx, word in enumerate(vocab):
      if word == w:
        bow[idx] = 1
  return np.array(bow)

def pred_class(text, vocab, labels):
  bow = bag_of_words(text, vocab)
  # Eksttraksi probabilitas
  result = model.predict(np.array([bow]))[0]
  thresh = 0.5 # sperti menyaring kata yang probabilitasnya di bawah 0.5 maka tidak akan di masukkan kedalam thres
  y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]
  # hasil prediksi di sort yang paling gede
  # x: x[1] setiap vocab akan di beri nomor dimuluai dari 1
  y_pred.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in y_pred:
    #berisi probbailitas terbesar untuk di gabung2 jd response
    return_list.append(labels[r[0]])
  return return_list

def get_response(intents_list, intents_json):
  if len(intents_list) == 0:
    result = "Sorry I don't understand"
  else:
    tag = intents_list[0]
    # mengambil list semua intent dari file intents.j
    list_of_intens = intents_json["intents"]
    for i in list_of_intens:
      if i["tag"] == tag:
        result = random.choice(i["responses"])
        break
  return result

In [ ]:
#Testing
print("press 0 untuk exit")
while True:
  print("masukan kata2 : ")
  message = input("")
  if message == "0":
    break
  intents = pred_class(message, words, classes)
  result = get_response(intents, data)
  print(result)

press 0 untuk exit
masukan kata2 : 
pagi
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Selamat pagi! Apa keluhan yang sedang kamu alami?
masukan kata2 : 
jagi ni pak saya pusing  panas batuk
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Pusing bisa disebabkan oleh kelelahan atau dehidrasi. Minumlah air dan hindari layar gadget sebentar.
masukan kata2 : 
batuk
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Batuk bisa karena infeksi atau alergi. Coba minum air hangat dan hindari makanan berminyak.
masukan kata2 : 
mual
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Cobalah duduk tenang dan konsumsi air hangat sedikit demi sedikit. Hindari makanan berat dulu ya.
masukan kata2 : 
oke dok terima kasih
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Sama-sama! Semoga cepat sembuh ya!
masukan kata2 : 
0
